# L-shaped algorithm example

We illustrate in this notebook the example 2 in Chapter 5, page 188, from Birge and Louveaux (2011), "Introduction to Stochastic Programming", 2nd edition, Springer.

In [ ]:
using StochasticPrograms
using HiGHS

First, we construct the 2-stage model.

In [ ]:
@stochastic_model simple_model begin
    @stage 1 begin
        @decision(simple_model, x >= 0)
        @objective(simple_model, Min, 0*x)
    end
    @stage 2 begin
        @known(simple_model, x)
        @uncertain ξ
        @recourse(simple_model, 0 <= y[i in 1:2])
        @objective(simple_model, Min, y[1] + y[2])
        @constraint(simple_model, y[1] - y[2] == ξ - x)
    end
end

We now define three scenarios of equal probability.

In [ ]:
ξ1 = @scenario ξ = 1 probability = 1/3
ξ2 = @scenario ξ = 2 probability = 1/3
ξ3 = @scenario ξ = 4 probability = 1/3

ξ = [ξ1, ξ2, ξ3]

## Deterministic equivalent

In order to have a solvable program, we must instantiate the model. By default, `StochasticPrograms.jl` will generate the derministic equivalent form.

In [ ]:
sp = instantiate(simple_model, ξ, optimizer = HiGHS.Optimizer)

We can check the model by printing it on the screen.

In [ ]:
println(sp)

We now solve it.

In [ ]:
optimize!(sp)

We can check the first stage solution with the method `value()`.

In [ ]:
value(sp[1,:x])

## L-shaped

We now express the program as a 2-stage optimization problem, and explore various variants of the L-shaped decomposition algorithm.

### Multi cut

In [ ]:
sp_lshaped = instantiate(simple_model, ξ, optimizer = LShaped.Optimizer)

In [ ]:
set_optimizer_attribute(sp_lshaped, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(sp_lshaped, SubProblemOptimizer(), HiGHS.Optimizer)

In [ ]:
optimize!(sp_lshaped)

The master problem is unbounded from below as $K_1$ is not bounded.

Let modify the first-stage feasible set by imposing a large bound on $x$, here $10^9$.

In [ ]:
@stochastic_model simple_model begin
    @stage 1 begin
        @decision(simple_model, 0 <= x <= 1e9)
        @objective(simple_model, Min, 0*x)
    end
    @stage 2 begin
        @known(simple_model, x)
        @uncertain ξ
        @recourse(simple_model, 0 <= y[i in 1:2])
        @objective(simple_model, Min, y[1] + y[2])
        @constraint(simple_model, y[1] - y[2] == ξ - x)
    end
end

In [ ]:
sp_lshaped = instantiate(simple_model, ξ, optimizer = LShaped.Optimizer)
set_optimizer_attribute(sp_lshaped, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(sp_lshaped, SubProblemOptimizer(), HiGHS.Optimizer)

println(sp_lshaped)

We now solve the program.

In [ ]:
optimize!(sp_lshaped)

The first-stage solution is

In [ ]:
value(sp_lshaped[1,:x])

### Single Cut

Let's check what happens when the single cut variant is used.

In [ ]:
set_optimizer_attribute(sp_lshaped, Aggregator(), Aggregate())
optimize!(sp_lshaped)

More iterations are needed, to obtain a similar first-stage solution.

In [ ]:
value(sp_lshaped[1,:x])